<a href="https://colab.research.google.com/github/Arsalna-Shaikh/colab-session-jupyter-nb-/blob/main/diarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. INSTALL DEPENDENCIES (RUN FIRST)
!pip install numpy==1.24.4  # Critical for compatibility
!pip install pyannote.audio==3.1.0 openai-whisper torch torchaudio
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# 2. AUTHENTICATE (RUN SECOND)
from huggingface_hub import notebook_login
notebook_login()  # Paste your Hugging Face token when prompted

In [ ]:
# 3. INITIALIZE PIPELINE (RUN THIRD)
import torchaudio
torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False  # Fix for torchaudio warning

from pyannote.audio import Pipeline
diarization = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=True
)

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

In [ ]:
# Run FOURTH
from google.colab import files
uploaded = files.upload()
audio_file = list(uploaded.keys())[0]

# Verify speakers are detected
for turn, _, speaker in diarization(audio_file).itertracks(yield_label=True):
    print(f"{speaker}: {turn.start:.1f}s → {turn.end:.1f}s")

Saving diarizatiion.waptt.opus to diarizatiion.waptt.opus


/usr/local/lib/python3.11/dist-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


SPEAKER_00: 1.3s → 12.5s
SPEAKER_00: 12.8s → 13.7s
SPEAKER_00: 14.8s → 15.0s
SPEAKER_01: 15.0s → 20.9s
SPEAKER_01: 21.3s → 26.5s
SPEAKER_00: 26.5s → 27.5s
SPEAKER_00: 28.2s → 38.2s
SPEAKER_01: 38.7s → 38.8s
SPEAKER_00: 38.8s → 38.9s
SPEAKER_01: 38.9s → 39.0s
SPEAKER_00: 39.0s → 39.0s
SPEAKER_01: 39.0s → 47.6s
SPEAKER_00: 48.6s → 55.2s
SPEAKER_00: 55.9s → 58.0s
SPEAKER_01: 58.0s → 59.0s
SPEAKER_00: 59.0s → 60.0s
SPEAKER_01: 60.0s → 62.1s


In [9]:
!ffmpeg -i diarizatiion.waptt.opus -ar 16000 -ac 1 converted.wav
audio_file = "converted.wav"  # Use this for all subsequent steps

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [10]:
import whisper

# Load Whisper model (use "small" for better accuracy)
model = whisper.load_model("base")

# Dictionary to store transcripts
speaker_transcripts = {}

for turn, _, speaker in diarization(audio_file).itertracks(yield_label=True):
    # Extract audio segment
    !ffmpeg -i {audio_file} -ss {turn.start} -to {turn.end} -c copy segment.wav -y

    # Transcribe
    result = model.transcribe("segment.wav")

    # Store results
    if speaker not in speaker_transcripts:
        speaker_transcripts[speaker] = []
    speaker_transcripts[speaker].append(result["text"])

    # Optional: Print progress
    print(f"Transcribed {speaker}'s segment ({turn.start:.1f}s-{turn.end:.1f}s)")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed SPEAKER_00's segment (1.3s-12.5s)
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvid

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed SPEAKER_00's segment (12.8s-13.7s)
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvi

...

Transcribed SPEAKER_01's segment (58.0s-59.0s)
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvi

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed SPEAKER_00's segment (59.0s-60.0s)
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvi

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed SPEAKER_01's segment (60.0s-62.1s)


In [11]:
import whisper
model = whisper.load_model("base")

speaker_transcripts = {}

for turn, _, speaker in diarization(audio_file).itertracks(yield_label=True):
    # Extract audio segment
    !ffmpeg -i {audio_file} -ss {turn.start} -to {turn.end} -c copy segment.wav -y

    # Transcribe and print immediately
    result = model.transcribe("segment.wav")
    print(f"\n{speaker} ({turn.start:.1f}s-{turn.end:.1f}s):")
    print(result["text"])  # <-- This shows the actual transcription

    # Store results
    if speaker not in speaker_transcripts:
        speaker_transcripts[speaker] = []
    speaker_transcripts[speaker].append(result["text"])

# Save to files (unchanged)
for speaker, texts in speaker_transcripts.items():
    with open(f"{speaker}.txt", "w") as f:
        f.write("\n\n".join(texts))
    files.download(f"{speaker}.txt")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



SPEAKER_00 (1.3s-12.5s):
 Hi, my name is Alex. I'm here to talk about artificial intelligence. Over the past few years, AI has made significant progress, especially in fields like healthcare, finance, and education.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-l

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



SPEAKER_00 (12.8s-13.7s):
 that slide.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab -

...


SPEAKER_01 (60.0s-62.1s):
 Sure, AI is here to see.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import whisper
model = whisper.load_model("base")

speaker_transcripts = {}

print("=== Speaker Transcripts ===")
for turn, _, speaker in diarization(audio_file).itertracks(yield_label=True):
    # Extract audio segment
    !ffmpeg -i {audio_file} -ss {turn.start} -to {turn.end} -c copy segment.wav -y

    # Transcribe
    result = model.transcribe("segment.wav")

    # Store and display
    if speaker not in speaker_transcripts:
        speaker_transcripts[speaker] = []
    speaker_transcripts[speaker].append(result["text"])

    # Print current segment
    print(f"\n{speaker} ({turn.start:.1f}s-{turn.end:.1f}s):")
    print(result["text"])

# Optional: Display consolidated transcripts at the end
print("\n=== Final Transcripts ===")
for speaker, texts in speaker_transcripts.items():
    print(f"\n{speaker}:")
    print("\n".join(texts))

=== Speaker Transcripts ===
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libv

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



SPEAKER_00 (1.3s-12.5s):
 Hi, my name is Alex. I'm here to talk about artificial intelligence. Over the past few years, AI has made significant progress, especially in fields like healthcare, finance, and education.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-l

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



SPEAKER_00 (12.8s-13.7s):
 that slide.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab -

...


SPEAKER_01 (58.0s-59.0s):
 evolved.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --en

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



SPEAKER_00 (59.0s-60.0s):
 one thing for
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



SPEAKER_01 (60.0s-62.1s):
 Sure, AI is here to see.

=== Final Transcripts ===

SPEAKER_00:
 Hi, my name is Alex. I'm here to talk about artificial intelligence. Over the past few years, AI has made significant progress, especially in fields like healthcare, finance, and education.
 that slide.

 instantly.
 But with great power comes with great responsibility. We need to make sure that AI systems are designed is physically and don't reinforce harmful biases.


 It's not easy, but ongoing research and explainable AI and transparent algorithms is a step in the right direction.
 Well, I guess we will see how it...
 one thing for

SPEAKER_01:
 One of the most exciting developments is the use of AI in real-time language translation.
 It's fascinating how a machine can now interrupt multiple languages almost.
 This is commling the

 But that's a big challenge. How do you even?
 Define fairness in AI when people have such different opinions.
 evolved.
 Sure, AI is here to see.


In [12]:
import whisper
model = whisper.load_model("base")

speaker_transcripts = {}

# Process audio segments
for turn, _, speaker in diarization(audio_file).itertracks(yield_label=True):
    !ffmpeg -i {audio_file} -ss {turn.start} -to {turn.end} -c copy segment.wav -y
    result = model.transcribe("segment.wav")

    if speaker not in speaker_transcripts:
        speaker_transcripts[speaker] = []
    speaker_transcripts[speaker].append(result["text"])

# Display results in separate paragraphs
print("=== Speaker 0 ===")
print("\n".join(speaker_transcripts.get("SPEAKER_00", ["No speech detected"])))
print("\n=== Speaker 1 ===")
print("\n".join(speaker_transcripts.get("SPEAKER_01", ["No speech detected"])))

In [13]:
for speaker, texts in speaker_transcripts.items():
    with open(f"{speaker}.txt", "w") as f:
        f.write("\n\n".join(texts))
    files.download(f"{speaker}.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install gradio pyannote.audio whisper torch

import gradio as gr
from pyannote.audio import Pipeline
import whisper
import tempfile
from google.colab import files

# Initialize models
diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token="YOUR_HUGGINGFACE_TOKEN"  # Replace with your token
)
transcription_model = whisper.load_model("base")

def process_audio(audio_file):
    # Save uploaded file
    audio_path = audio_file.name

    # Diarization
    diarization = diarization_pipeline(audio_path)

    # Process each speaker segment
    speaker_texts = {}
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        # Extract segment
        with tempfile.NamedTemporaryFile(suffix='.wav') as temp_file:
            !ffmpeg -i {audio_path} -ss {turn.start} -to {turn.end} -c copy {temp_file.name} -y

            # Transcribe
            result = transcription_model.transcribe(temp_file.name)

            if speaker not in speaker_texts:
                speaker_texts[speaker] = []
            speaker_texts[speaker].append(result["text"])

    # Format output
    output = ""
    for speaker, texts in speaker_texts.items():
        output += f"=== {speaker} ===\n"
        output += "\n".join(texts) + "\n\n"

    return output

# Create Gradio interface
interface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath", label="Upload Audio"),
    outputs=gr.Textbox(label="Speaker Transcripts", lines=10),
    title="Speaker Diarization & Transcription",
    description="Upload an audio file to separate speakers and transcribe their dialogue"
)

# For Colab
def start_interface():
    import nest_asyncio
    nest_asyncio.apply()
    interface.launch()

start_interface()